In [9]:
import os
import json
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import pandas as pd


In [14]:
def layer_alterator(
    raster_folder,
    vector_path,
    operation_rule_path,
    output_folder=None,
    value_range=(0, 1)
):
    """
    Modifies raster pixels using vector-defined zones and attribute-based operations.

    Parameters:
    - raster_folder (str): Folder containing rasters (supports subfolders).
    - vector_path (str): Path to vector mask (.geojson or .gpkg).
    - operation_rule_path (str): Path to JSON file mapping raster filenames to operation rules ('pct', 'replace', or None).
    - output_folder (str): Where to save the results.
    - value_range (tuple): Min/max clip range for pixel values.
    """

    # Load operation rules from JSON
    with open(operation_rule_path, 'r') as f:
        operation_rules = json.load(f)

    mask_gdf = gpd.read_file(vector_path)

    # Collect all .tif files recursively
    raster_files = []
    for root, dirs, files in os.walk(raster_folder):
        for file in files:
            if file.lower().endswith((".tif", ".tiff")):
                raster_files.append(os.path.join(root, file))

    for raster_path in raster_files:
        raster_filename = os.path.basename(raster_path)
        raster_name = os.path.splitext(raster_filename)[0]
        attribute_field = raster_name.upper()

        # Determine operation rule for this raster
        operation_rule = operation_rules.get(raster_filename)

        if operation_rule is None:
            print(f"[SKIP] No operation rule for '{raster_filename}'")
            continue

        if attribute_field not in mask_gdf.columns:
            print(f"[SKIP] Attribute '{attribute_field}' not found in vector data.")
            continue

        with rasterio.open(raster_path) as src:
            raster_data = src.read(1)
            transform = src.transform
            crs = src.crs
            dtype = src.dtypes[0]
            nodata = src.nodata

            for _, row in mask_gdf.iterrows():
                if pd.isna(row.get(attribute_field)):
                    continue

                try:
                    value = float(row[attribute_field])
                except (ValueError, TypeError):
                    print(f"[SKIP] Invalid value for '{attribute_field}': {row[attribute_field]}")
                    continue

                geometry = [row["geometry"]]

                # Mask the raster using the current geometry
                masked_data, _ = mask(src, geometry, crop=False)
                masked_data = masked_data[0]

                # Create the mask of affected pixels
                if nodata is not None:
                    polygon_mask = masked_data != nodata
                else:
                    polygon_mask = masked_data != 0  # fallback

                # Apply the operation
                if operation_rule == "pct":
                    modified = masked_data * (1 - value / 100.0)
                elif operation_rule == "replace":
                    modified = np.full_like(masked_data, value)
                else:
                    raise ValueError(f"Unknown operation_rule '{operation_rule}' for '{raster_filename}'. Use 'pct', 'replace', or null.")

                modified = np.clip(modified, *value_range)
                raster_data[polygon_mask] = modified[polygon_mask]

        # Save the modified raster
        if output_folder:
            os.makedirs(output_folder, exist_ok=True)
            output_path = os.path.join(output_folder, raster_filename)
        else:
            output_path = f"modified_{raster_filename}"

        with rasterio.open(
            output_path, "w", driver="GTiff",
            height=raster_data.shape[0], width=raster_data.shape[1],
            count=1, dtype=dtype, crs=crs, transform=transform
        ) as dst:
            dst.write(raster_data, 1)

        # Correctly indented print
        print(f"[DONE] Modified {raster_filename} with '{operation_rule}' → {output_path}")

In [15]:
layer_alterator(
    raster_folder="./test_data/test_data_final/lc_fractions",
    vector_path="./test_data/test_data_final/sample_mask.geojson",
    operation_rule_path="./test_data/test_data_final/operation_rules.json",
    output_folder="./output/modified_raster/new_data",
    value_range=(0, 1)
)

[SKIP] No operation rule for 'F_W.tif'
[DONE] Modified F_S.tif with 'replace' → ./output/modified_raster/new_data/F_S.tif
[SKIP] No operation rule for 'F_G.tif'
[SKIP] No operation rule for 'F_BS.tif'
[SKIP] No operation rule for 'F_TV.tif'
[DONE] Modified F_AC.tif with 'replace' → ./output/modified_raster/new_data/F_AC.tif
[DONE] Modified F_M.tif with 'replace' → ./output/modified_raster/new_data/F_M.tif
